Import environment variables

In [2]:
from dotenv import load_dotenv
import os

load_dotenv('.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

Load our document corpus from a file. (fetch_data.ipynb can be used to generate the file)

In [3]:
myfile = "source_documents.json"

import json
from langchain.schema import Document

# Load JSON data
with open(myfile, 'r') as file:
    data = json.load(file)

# Convert JSON data into a list of LangChain Document objects
docs = [
    Document(page_content=item["page_content"], metadata=item["metadata"])
    for item in data
]

print(f"loaded {len(docs)} docs")

loaded 438 docs


Split the documents into reasonably sized chunks that work for most embedding models

In [127]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,       
    chunk_overlap=150,     
)

n_webmd_docs = 0 
split_docs = []

for doc in docs:
    if "webmd" in doc.metadata["url"]:
        n_webmd_docs += 1

    splits = text_splitter.split_text(doc.page_content)
    for i,split in enumerate(splits):
        metadata_with_chunk = {**doc.metadata, "chunk_id": i}
            
        # Create the document with the updated metadata
        split_doc = Document(page_content=split, metadata=metadata_with_chunk)
        split_docs.append(split_doc)

print(f"webmd docs: {n_webmd_docs}")
print(f"len(docs): {len(docs)}, len(split_docs):{len(split_docs)}")
print(split_docs[0])

webmd docs: 242, videos,forms and slideshows: 0
len(docs): 438, len(split_docs):2561
page_content='alzheimer's disease and dementia | alzheimer's disease and dementia | cdc     alzheimer's disease and dementia alzheimer's basics learn about signs and symptoms of alzheimer's disease and who is affected. aug. 15, 2024 dementia basics learn about common types of dementia, signs and symptoms, and risk factors. aug. 17, 2024 signs and symptoms of alzheimer's learn how to recognize the early signs of alzheimer's disease. signs and symptoms of dementia learn what early signs and symptoms of dementia to look out for. tools and resources find a variety of resources about alzheimer’s disease and healthy aging. reducing risk learn what lifestyle behaviors can reduce the risk of developing dementia. additional topics healthy aging at any age information to help you stay healthy and strong throughout your life. sept. 3, 2024 alzheimer's disease program evidence-based, scientific information to educ

Set up embeddings - we'll use OpenAI's text-embedding-3-large

In [6]:
from langchain_openai import OpenAIEmbeddings
embedding_model = "text-embedding-3-large"
openai_embeddings = OpenAIEmbeddings(
    model=embedding_model,
    openai_api_key=OPENAI_API_KEY  
)


Let's add the docs to a vector store. Make sure qdrant is running first (see README.md for more details). We can create it once and re-use it after that.

In [150]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

url="http://localhost:6333"
collection_name = "AlzheimersCare"

client = QdrantClient(url=url, prefer_grpc=True)
client.delete_collection(collection_name)

qdrant_vector_store = QdrantVectorStore.from_documents(
    split_docs,
    openai_embeddings,
    url=url,
    prefer_grpc=True,
    collection_name=collection_name,
)

Make sure we can load the vectorstore too

In [8]:
# make sure we can load it
from langchain_qdrant import QdrantVectorStore
collection_name = "AlzheimersCare"
url="http://localhost:6333"


store = QdrantVectorStore.from_existing_collection(
    embedding=openai_embeddings,
    collection_name=collection_name,
    url=url
)

Test it out by itself

In [9]:
from langchain.retrievers import EnsembleRetriever

mmr_retriever = store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'lambda_mult': 0.1}
)

similarity_retriever = store.as_retriever(k=10)

retriever = EnsembleRetriever(retrievers=[mmr_retriever,similarity_retriever],id_key="url")
results = retriever.invoke("How does stress impact dementia caregivers?")

for result in results: print(result)

page_content='strain may develop from the amount of time your loved one needs, as that can impact your ability to work and earn a living. things like lost wages, or loss of social security or pension benefits, can leave caregivers struggling to make ends meet. your risk for mental health issues also goes up. research shows that alzheimer’s caregivers are more likely than other caregivers to have depression, anxiety, and a poorer quality of life. among the physical health problems you’re at greater risk of having are high blood pressure, high blood sugar, weight gain or loss, and sleep disorders. some research suggests you’re also at greater risk for developing cognitive decline and alzheimer’s yourself. caregivers at most risk for reaching stress levels that impact their well-being are: caring for a person with alzheimer’s for many hours each day living with the person for whom they’re the caregiver women older socially isolated having financial problems clinically depressed at a low e

Test it out in a simple RAG chain: Create a prompt, initialize an LLM, and then use the retriever in a chain

In [10]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """
You are an empathetic, kind assistant that specializes in helping informal caregivers of dementia and Alzheimer's patients
navigate the stresses and questions of everyday life. Answer the question based on the context. If the answer is not
in the context, say you don't know. Be concise and conversational, and answer in language that a high school 
graduate with no specialized training can understand. 

You must never give medical, legal, or financial advice. Always make sure the 
user contacts a professional if it is an emergency or if they need medical advice.

<context>
{context}
</context>

<question>
{query}
<question>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [11]:
from langchain_anthropic import ChatAnthropic

haiku_model_id = "claude-3-haiku-20240307" # cheaper and better to use for prototyping, although we'll use 3.5 in our app
claude_3_5_sonnet_model_id = "claude-3-5-sonnet-20240620"

llm = ChatAnthropic(
    model=haiku_model_id,    
    anthropic_api_key=ANTHROPIC_API_KEY
)

In [12]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# prototype a simple function to tack on sources at the end
def add_sources(context:list[Document])->str:
    sources_str = ""
    if len(context)>0:
        i = 1
        sources_str = "Sources: "

        for doc in context:
            if not doc.metadata.get("url") in sources_str:
                sources_str += f'[<a href="{doc.metadata["url"]}">{i}</a>] '
                i+=1

    return sources_str

# standard RAG that passes the context through
max_context = 4
rag_chain = (
    {"context": itemgetter("query") | retriever | (lambda docs: docs[:max_context]), "query": itemgetter("query")} 
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [13]:
async def answer_question(user_input):
    answer = await rag_chain.ainvoke(input={'query':user_input})
    answer["response"].pretty_print()
    print(add_sources(answer["context"]))   
    return answer["response"],answer["context"]


In [14]:
_, context = await answer_question("who is a typical caregiver?")

================================== Ai Message ==================================

Based on the context provided, a typical caregiver for someone with Alzheimer's or dementia is:

- Often a family member, such as a spouse or adult child, who is taking on the responsibility of caring for their loved one as the disease progresses.
- Balancing caregiving duties with other responsibilities like a job and taking care of their own family.
- Facing the challenges of seeing their loved one's decline in abilities and independence over time.
- Struggling with feelings of guilt, sleeplessness, and the solitary nature of the caregiving role.
- Needing to build a network of support and help to manage the increasing demands of caregiving.

The context does not provide information about a "typical" caregiver in terms of demographics or other specifics, so I cannot say anything definitive beyond the points above based on the information given.
Sources: [<a href="https://www.webmd.com/alzheimers/feature

In [15]:
await answer_question("i found an old man walking by the side of the road and he doesn't remember anything, what should i do?")
await answer_question("what are some early warning signs of dementia?")
await answer_question("my mom is having a bad day and wants the car keys, what do i do?")

================================== Ai Message ==================================

Here's what you can do if you encounter an older adult who seems confused or lost and may have dementia:

1. Stay calm and approach the person gently. Identify yourself and ask if they need help. Speak slowly and clearly.

2. Try to gather information from the person, such as their name, where they live, or if they have any emergency contacts. This can help you locate their family or caregivers.

3. If the person is unable to provide helpful information, call the police non-emergency number. Explain the situation and provide a description of the person so they can try to locate their home or caregivers.

4. Do not try to forcibly take the person somewhere or drive them yourself, as this could make them more distressed. Wait with them if possible until the police or their caregiver arrives.

5. Avoid arguing or correcting the person if they seem disoriented. Be patient and compassionate.

The most importan

(AIMessage(content='I\'m sorry to hear your mom is having a difficult day. Dealing with a loved one\'s request to drive when it may not be safe can be really challenging. Here are a few suggestions that may help:\n\nFirst, try not to argue or get confrontational. Validate her feelings by saying something like "I understand you want to go out and that must be frustrating." Then gently explain your concerns about her driving, focusing on her safety and wellbeing. You could say something like "I\'m worried about you driving right now because I want to make sure you get where you need to go safely."\n\nSuggest alternatives, like offering to drive her yourself or looking into transportation options like senior shuttles or ride-sharing services in your area. Emphasize that you want to help her maintain her independence as much as possible.\n\nIf she continues to insist on driving, don\'t physically take the keys. Instead, you could hide them or say they can\'t be found. The goal is to avoid 